In [2]:
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils
from sklearn.model_selection import train_test_split


In [3]:
root_directory = os.getcwd()

working_directory = f"{root_directory}/working/frames/"
json_directory = f"{root_directory}/working/json/"
face_directory = f"{root_directory}/working/faces/"

# Load the FER2013 dataset
data = pd.read_csv(f"{root_directory}/utils/prebuilt/fer2013.csv")

In [5]:
import cv2

In [6]:
train_data = data[data['Usage'] == 'Training']
test_data = data[data['Usage'] == 'PrivateTest']                  
val_data = data[data['Usage'] == 'PublicTest']                  

In [7]:
new_size = (128,128)

In [8]:
train_data


,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
28704,2,84 85 85 85 85 85 85 85 86 86 86 87 86 86 91 9...,Training
28705,0,114 112 113 113 111 111 112 113 115 113 114 11...,Training
28706,4,74 81 87 89 95 100 98 93 105 120 127 133 146 1...,Training
28707,0,222 227 203 90 86 90 84 77 94 87 99 119 134 14...,Training


In [9]:
# Reshape and resize the training set
x_train = np.array([cv2.resize(np.array(image.split(' '), dtype='uint8').reshape(48, 48), new_size).reshape(128, 128, 1) for image in train_data['pixels']])

In [10]:
x_test = np.array([cv2.resize(np.array(image.split(' '), dtype='uint8').reshape(48, 48), new_size).reshape(128, 128, 1) for image in test_data['pixels']])

In [11]:
x_val = np.array([cv2.resize(np.array(image.split(' '), dtype='uint8').reshape(48, 48), new_size).reshape(128, 128, 1) for image in val_data['pixels']])

In [12]:
y_train = np.array(train_data['emotion'])


In [13]:
y_test = np.array(train_data['emotion'])


In [14]:
y_val = np.array(train_data['emotion'])

In [15]:
x_train.shape

(28709, 128, 128, 1)

In [16]:
x_test

array([[[[170],
         [167],
         [147],
         ...,
         [127],
         [124],
         [124]],

        [[170],
         [167],
         [147],
         ...,
         [127],
         [125],
         [125]],

        [[169],
         [166],
         [148],
         ...,
         [128],
         [128],
         [128]],

        ...,

        [[101],
         [100],
         [102],
         ...,
         [131],
         [131],
         [131]],

        [[104],
         [103],
         [103],
         ...,
         [132],
         [131],
         [131]],

        [[104],
         [104],
         [103],
         ...,
         [132],
         [131],
         [131]]],


       [[[  7],
         [  7],
         [  6],
         ...,
         [ 13],
         [ 15],
         [ 15]],

        [[  7],
         [  7],
         [  6],
         ...,
         [ 13],
         [ 15],
         [ 15]],

        [[  7],
         [  7],
         [  6],
         ...,
         [ 13],
         [

In [17]:
x_train = x_train.astype('float32') / 255.0


In [18]:
x_test = x_train.astype('float32') / 255.0


In [19]:
x_val = x_train.astype('float32') / 255.0

In [20]:
num_classes = len(np.unique(y_train))
num_classes

7

In [21]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6])

In [22]:
y_train = np.eye(num_classes)[y_train]
y_test = np.eye(num_classes)[y_test]
y_val = np.eye(num_classes)[y_val]

In [23]:

# Define the model architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))



2023-04-20 19:30:12.310473: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-04-20 19:30:12.311881: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (blacklisted): /proc/driver/nvidia/version does not exist
2023-04-20 19:30:12.317871: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-20 19:30:13.540359: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 125960192 exceeds 10% of free system memory.
2023-04-20 19:30:13.598848: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 125960192 exceeds 10% of free sys

In [25]:
model.summary

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x7f38f2771e40>>

In [27]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      320       
                                                                 
 conv2d_1 (Conv2D)           (None, 124, 124, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 62, 62, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 62, 62, 64)        0         
                                                                 
 flatten (Flatten)           (None, 246016)            0         
                                                                 
 dense (Dense)               (None, 128)               31490176  
                                                        

In [31]:
model.fit(x_train,y_train, batch_size=32, epochs=10, validation_data=(x_val,y_val))

Epoch 1/10


2023-04-20 19:34:53.620488: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1881473024 exceeds 10% of free system memory.
2023-04-20 19:34:56.016390: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 125960192 exceeds 10% of free system memory.


898/898 [==============================] - 634s 705ms/step - loss: 1.7647 - accuracy: 0.3010 - val_loss: 2.0703 - val_accuracy: 0.1682
Epoch 2/10
898/898 [==============================] - 630s 701ms/step - loss: 1.5856 - accuracy: 0.3828 - val_loss: 2.0235 - val_accuracy: 0.1682
Epoch 3/10
898/898 [==============================] - 612s 682ms/step - loss: 1.4999 - accuracy: 0.4145 - val_loss: 2.3906 - val_accuracy: 0.1682
Epoch 4/10
898/898 [==============================] - 613s 683ms/step - loss: 1.4115 - accuracy: 0.4523 - val_loss: 2.1873 - val_accuracy: 0.1682
Epoch 5/10
898/898 [==============================] - 611s 681ms/step - loss: 1.3163 - accuracy: 0.4922 - val_loss: 2.1351 - val_accuracy: 0.1682
Epoch 6/10
898/898 [==============================] - 611s 681ms/step - loss: 1.2064 - accuracy: 0.5375 - val_loss: 2.1196 - val_accuracy: 0.1682
Epoch 7/10
395/898 [============>.................] - ETA: 5:05 - loss: 1.0865 - accuracy: 0.5839